# PyTorch basics

To step through this notebook, you will need to install a number of packages. First off all, we will need PyTorch, please follow the installation instructions provided at http://pytorch.org/.

After successfully doing so, install the following additional packages:
- `visdom` : package for visualisations
- `tqdm` : package to display progress bars

(Use `conda` or `pip` / `pip3` depending on your local setup.)

You can skip execution of the next cell. 

(Executing the next cell enables presentation mode (navigate with arrow keys in cell mode); to get out of presentation mode, clear all cell output -- the menu becomes visible on hover)

In [ ]:
%%html
<link rel="stylesheet" href="css/jupyter.css">
<link rel="stylesheet" href="css/presenter.css">
<link rel="stylesheet" href="css/cells.css">
<link rel="stylesheet" href="css/codemirror.css">

In [ ]:
# package imports

# numpy
import numpy as np
np.random.seed(42)

# progress bars
from tqdm import tqdm_notebook as tqdm

# in case GPUs are used, limit to single device
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# PyTorch imports
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from torch import nn
from torch.autograd import Variable
from torchvision import transforms

dtype = torch.FloatTensor
# dtype = torch.cuda.FloatTensor  # Uncomment this to run on GPU

# matplotlib for plotting
import matplotlib.pyplot as plt
fig_size = (7, 7)
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['figure.figsize'] = fig_size
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'none'
plt.rcParams['xtick.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.right'] = False
plt.rcParams['ytick.color'] = 'white'
%matplotlib inline

# widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# visdom
import visdom

<span class="big"><b>Intro to PyTorch</span>

<br/>

<h2 style="line-height: 1.4em; font-size: 1.7em;">MIE Deep Learning Bootcamp, Berlin 2018</h2>

<img src='./img/pytorch_front.png' width=100%>

<img src='./img/pytorch_users.png' width=100%>

<img src='./img/pytorch_google.png'>

# Tensor computation on GPU
# Deep learning and automatic differentiation
# Optimizers, data loading utilities

<img src='./img/pytorch_pkg.png'>

# PyTorch as a fast calculator

In [ ]:
import numpy as np

d = 3000

# compute Z = X Y
X = np.random.rand(d, d).astype(np.float32)
Y = np.random.rand(d, d).astype(np.float32)
Z = X.dot(Y)

# time
%timeit -n1 -r10 Z = X.dot(Y)

In [ ]:
import torch

d = 3000

# compute Z = X Y
X = torch.rand(d, d).type(torch.FloatTensor)
Y = torch.rand(d, d).type(torch.FloatTensor)
Z = torch.mm(X, Y)

# time
%timeit -n1 -r10 Z = torch.mm(X, Y)

In [ ]:
import tensorflow as tf

d = 3000

# static graph
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)
Z = tf.matmul(X, Y)

# values to feed
X_np = np.random.rand(d, d).astype(np.float32)
Y_np = np.random.rand(d, d).astype(np.float32)

# create session and feed values
with tf.Session() as sess:
    sess.run(Z, feed_dict={X: X_np, Y: Y_np})
    
    # time
    %timeit -n1 -r10 sess.run(Z, feed_dict={X: X_np, Y: Y_np})

In [ ]:
import torch

d = 3000

# compute Z = X Y
X = torch.rand(d, d).type(torch.FloatTensor)
Y = torch.rand(d, d).type(torch.FloatTensor)
Z = torch.mm(X, Y)

# time
%timeit -n1 -r10 Z = torch.mm(X, Y)

In [ ]:
import torch

d = 3000

if torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor
else:
    dtype = torch.FloatTensor

X = torch.rand(d, d).type(dtype)
Y = torch.rand(d, d).type(dtype)
Z = torch.mm(X, Y)

%timeit -n1 -r10 Z = torch.mm(X, Y)

<img src='./img/ref_mm.png'>

In [ ]:
X.mm(Y)

<img src='./img/ref_matmul.png'>

In [ ]:
%%timeit 
a = np.zeros(10000000)
b = a * 2

In [ ]:
%%timeit 
a = np.zeros(10000000)
a *= 2

In [ ]:
%%timeit 
a = torch.zeros(10000000)
b = a * 2

In [ ]:
%%timeit 
a = torch.zeros(10000000)
a.mul_(2)

<span class='big'>PyTorch for automatic differentiation</span>

In [ ]:
import torch

x = torch.arange(1, 6)
x

In [ ]:
x = torch.arange(1, 6)

# Task: compute d(||x||^2)/dx

$$f(\mathbf{x}) = ||\mathbf{x}||^2 = \mathbf{x}^T\mathbf{x}$$

here: $$1^2 + 2^2 + 3^2 + 4^2 + 5^2 = 55$$

In [ ]:
from torch.autograd import Variable

> A PyTorch Variable is a wrapper around a PyTorch Tensor, and represents a node in a computational graph. <span class='highlight'>If x is a Variable then x.data is a Tensor giving its value, and x.grad is another Variable holding the gradient of x with respect to some scalar value.</span>
> 
> PyTorch Variables have the same API as PyTorch tensors: (almost) any operation you can do on a Tensor you can also do on a Variable; the difference is that autograd allows you to automatically compute gradients.


In [ ]:
# Task: compute d(||x||^2)/dx

#x = torch.arange(1, 6)
x = Variable(torch.arange(1, 6), requires_grad=True)

x.data

In [ ]:
x.grad is None

In [ ]:
# Task: compute d(||x||^2)/dx

x = Variable(torch.arange(1, 6), requires_grad=True)

f = x.dot(x)

f.data

$$f(\mathbf{x}) = ||\mathbf{x}||^2 = \mathbf{x}^T\mathbf{x}$$

here: $$1^2 + 2^2 + 3^2 + 4^2 + 5^2 = 55$$

derivative of $f(\mathbf{x})$ wrt to $\mathbf{x}$:

$$[2 x_1, 2 x_2, 2 x_3, 2 x_4, 2 x_5]$$

here: $$[ 2, 4, 6, 8, 10 ]$$

In [ ]:
# Task: compute d(||x||^2)/dx

x = Variable(torch.arange(1, 6), requires_grad=True)

f = x.dot(x)

f.backward()

x.grad

![](img/eyes.jpg)

http://www.npr.org/sections/health-shots/2015/08/07/430149677/eye-shapes-of-the-animal-world-hint-at-differences-in-our-lifestyles

In [ ]:
from scipy.ndimage import imread

eyes = imread('img/eyes.jpg', mode='RGB')
eye_img = eyes[:425, :425, :]  # width, height, depth
plt.imshow(eye_img);

img_width, img_height, img_depth = eye_img.shape

In [ ]:
input_img_np = eye_img  # W x H x C
input_img_np = np.swapaxes(input_img_np, 0, 2)  # C x H x W
input_img_np = input_img_np.reshape(1, img_depth, img_height, img_width)

print(input_img_np.shape)

In [ ]:
dtype = torch.FloatTensor
#dtype = torch.cuda.FloatTensor

input_img = torch.from_numpy(input_img_np).type(dtype)

input_img = Variable(input_img)

print(input_img.size())  # B x C x H x W

# 2D convolution

![](img/no_padding_no_strides.gif)

Dumoulin & Visin, 2016

In [ ]:
kernel_np = np.array([[-1,  0,  1],
                      [ 0,  0,  0],
                      [ 1,  0, -1]])
kernel_np = np.asarray((kernel_np, 
                     kernel_np, 
                     kernel_np))
kernel_np = kernel_np[np.newaxis, :, :, :]  # K, C, H, W

print(kernel_np.shape)

In [ ]:
kernel = torch.from_numpy(kernel_np).type(dtype)
kernel = Variable(kernel)

print(kernel.size())

In [ ]:
conv_edge = torch.nn.functional.conv2d(input_img, 
                                       kernel, 
                                       stride=1)

print(conv_edge.data)

In [ ]:
result = conv_edge.data.numpy().T  # B, C, H, W  ->  W, H, C, B

plt.figure(figsize=(9,9))

plt.subplot(1,2,1)
plt.title(eye_img.shape)
plt.imshow(eye_img);

plt.subplot(1,2,2)
plt.title(result.shape[:3])
plt.imshow(result.squeeze());

In [ ]:
conv1 = nn.Conv2d(in_channels=3, 
                  out_channels=1, 
                  kernel_size=3, 
                  stride=1, 
                  padding=0, 
                  bias=False)

plt.imshow(conv1(input_img).data.numpy().T.squeeze());

![](img/def_init.png)

https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/conv.py#L40,L47

In [ ]:
conv1.weight

In [ ]:
net = nn.Sequential(conv1, 
                    #nn.ReLU()
                   )

plt.imshow(net(input_img).data.numpy().T.squeeze());

In [ ]:
net(input_img)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=1e-7, momentum=0.9)

for i in range(100):
    optimizer.zero_grad()
    
    pred = net(input_img)

    loss = criterion(pred, conv_edge)
    loss.backward()

    optimizer.step()

    if i % 10 == 0:
        print(loss.data[0])

In [ ]:
plt.imshow(net(input_img).data.numpy().T.squeeze());

In [ ]:
print('Kernel : {}'.format(kernel))

print('Learned : {}'.format(next(net.parameters())))

## visdom

![](img/visdom_anim.gif?rnd={np.random.rand(})

Start server:

```bash
python -m visdom.server -port 9000
```

Then open:

http://localhost:9000

In [ ]:
import visdom

vis = visdom.Visdom(port=9000)

In [ ]:
vline = vis.line(X=np.linspace(-10., 10., 300),
                 Y=np.random.randn(300))

In [ ]:
conv1.reset_parameters()

batch = 0
vline = vis.line(X=np.empty((1)), Y=np.empty((1)))

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=1e-7, momentum=0.9)

for b in tqdm(range(10000)):
    optimizer.zero_grad()
    
    pred = net(input_img)

    loss = criterion(pred, conv_edge)
    loss.backward()

    optimizer.step()

    batch += 1
    vis.updateTrace(X=np.asarray([batch]), 
                    Y=np.asarray([loss.data[0]]), 
                    win=vline) 

In [ ]:
torch.manual_seed(42)

if args.cuda:
    torch.cuda.manual_seed(42)

![](img/ref_mnist.png)